In [1]:
import numpy as np
from numpy import nan
import pandas as pd

In [39]:
key = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'conversions')
df = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'raw data')

def create_dict_key(dataframe):
    '''
    '''
    conversion_key = dict()
    for year in dataframe.columns[1:]:
        values = list(dataframe[year])
        categories = list(dataframe['categories'])
        temp = dict()
        for tup in tuple(zip(categories, values)):
            temp[tup[0]] = tup[1]
        
        temp['USD'] = 1
        conversion_key[year] = temp
            
    return conversion_key

conversion_key = create_dict_key(key)


def adjustments(raw_data, conversion_key, fy_starts=13):
    '''
    Takes inputs of raw_data and conversion key and
    converts to real money terms.
    
    Inputs:
        raw_data: pandas dataframe of raw data Excel
        raw_data: pandas dataframe of raw data Excel
        
    Outputs:
        adjusted_data: pandas dataframe of adjusted data
    '''
    columns = list(raw_data.columns[fy_starts:])
    raw_data['Region'] = raw_data['Type']
        
    for index, row in raw_data.iterrows():
        currency = row['Currency']
        for year in columns:
            year_match = int(year[2:])
            infl = conversion_key[year_match]['Inflation Adjustment']
            curr = conversion_key[year_match][currency]
            if currency == 'Yen':
                row[year] = (float(row[year])/(curr)*1000)*infl
            else:
                row[year] = (float(row[year])/curr)*infl
            
        raw_data.loc[index] = row
        
    clean = raw_data
    clean['Source'] = clean['Source'].str.replace(" ", "")
    
    return clean[['Proper Name', 'Source'] + columns]

clean = adjustments(df, conversion_key)

In [42]:
clean[clean['Proper Name'] == 'Gilenya']

,Proper Name,Source,FY2007,FY2008,FY2009,FY2010,FY2011,FY2012,FY2013,FY2014,...,FY2029,FY2030,FY2031,FY2032,FY2033,FY2034,FY2035,FY2036,FY2037,FY2038
57,Gilenya,Gilenya1,0,0.0,0.0,0.0,650.104,1545.3740,2470.4916,3122.7539,...,149.181564,111.186538,86.991129,28.818214,24.702470,21.395462,18.705015,NaN,NaN,NaN
58,Gilenya,Gilenya2,0,0.0,0.0,0.0,504.028,940.1564,1306.7802,1500.2330,...,59.073645,45.173891,39.067537,7.198548,6.253689,5.492997,4.884443,NaN,NaN,NaN


In [36]:
df = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'raw data')
df

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Manufacturer,Look Company,Look Up Revenue Line,Type,Source,Currency,...,FY2029,FY2030,FY2031,FY2032,FY2033,FY2034,FY2035,FY2036,FY2037,FY2038
0,Actemra,Tocilizumab,2.821445e+08,Genentech,Roche,ROG_CH,Actemra,Total,Actemra 1,Franc,...,976.61,926.59,982.63,1025.78,1066.75,1060.30,1243.33,NaN,NaN,NaN
1,Actemra,Tocilizumab,2.821445e+08,Genentech,Roche,ROG_CH,Actemra US,US,Actemra 2,Franc,...,319.26,307.68,334.71,342.81,362.47,358.03,362.96,NaN,NaN,NaN
2,Adempas,Riociguat,4.631524e+08,Bayer,Bayer,BAYGN_DE,Adempas (Riociguat),P1Total,Adempas 1,Euro,...,325.92,331.98,298.12,126.24,104.09,86.95,38.50,NaN,NaN,NaN
3,Adempas,Riociguat,4.631524e+08,Bayer,Merck,MRK,International Adempas,P2ExUS,Adempas 2,USD,...,222.05,180.50,118.70,120.00,119.73,38.45,26.92,NaN,NaN,NaN
4,Adempas,Riociguat,4.631524e+08,Bayer,Merck,MRK,US Adempas,P2US,Adempas 3,USD,...,240.14,227.59,256.50,250.00,249.94,71.66,50.16,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Xtandi,Enzalutamide,1.968568e+09,Medivation,Pfizer/Astellas,Reports,Xtandi Global (Astellas Report),P2Total,Xtandi 4,Yen,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0
214,Xtandi,Enzalutamide,1.968568e+09,Medivation,Pfizer/Astellas,Reports,Xtandi US (Astellas Report),P2US,Xtandi 5,Yen,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0
215,Xtandi,Enzalutamide,1.968568e+09,Medivation,Pfizer/Astellas,Reports,Xtandi US (Astellas Report),P2US,Xtandi 6,USD,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0
216,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BMS,BMY,Revenue Yervoy,Total,Yervoy 1,USD,...,1250.75,1046.87,661.07,563.41,452.92,768.71,750.19,NaN,NaN,NaN


In [4]:
conversion_key

{2007: {'Inflation Adjustment': 1.4165,
  'Euro': 1.37,
  'Franc': 1.2,
  'Krone': 5.444928,
  'Pound': 2.0,
  'Yen': 117.76,
  'USD': 1},
 2008: {'Inflation Adjustment': 1.4015,
  'Euro': 1.47,
  'Franc': 1.08,
  'Krone': 5.092571,
  'Pound': 1.85,
  'Yen': 103.39,
  'USD': 1},
 2009: {'Inflation Adjustment': 1.3762,
  'Euro': 1.39,
  'Franc': 1.08,
  'Krone': 5.35371,
  'Pound': 1.57,
  'Yen': 93.68,
  'USD': 1},
 2010: {'Inflation Adjustment': 1.3606,
  'Euro': 1.33,
  'Franc': 1.04,
  'Krone': 5.621846,
  'Pound': 1.55,
  'Yen': 87.78,
  'USD': 1},
 2011: {'Inflation Adjustment': 1.316,
  'Euro': 1.39,
  'Franc': 0.89,
  'Krone': 5.35608,
  'Pound': 1.6,
  'Yen': 79.7,
  'USD': 1},
 2012: {'Inflation Adjustment': 1.2932,
  'Euro': 1.29,
  'Franc': 0.94,
  'Krone': 5.793938,
  'Pound': 1.59,
  'Yen': 79.822,
  'USD': 1},
 2013: {'Inflation Adjustment': 1.2774,
  'Euro': 1.33,
  'Franc': 0.93,
  'Krone': 5.617573,
  'Pound': 1.56,
  'Yen': 97.6,
  'USD': 1},
 2014: {'Inflation Adjust

In [3]:
key = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'dictionary')

def split_strings(string):
    '''
    '''
    new_string = '{'
    if string == '':
        return string
    for char in string:
        if char not in ['+', '-', '*', '/', '(', ')']:
            new_string += char
        else:
            new_string += '}'
            new_string += char
            new_string += '{'
    
    new_string += '}'
    new_string = new_string.replace("{}", "" )
    
    return new_string.replace("{0.5}", "0.5" )


def formulas(dataframe):
    '''
    '''
    dataframe["formula"] = dataframe["formula"].apply(split_strings)
    formula_key = dict()
    for _, row in dataframe.iterrows():
        formula = row['formula']
        formula_key[row['ID']] = formula

    return formula_key

formula_key = formulas(key)

In [28]:
formula_key

{'Actemra WW': '{Actemra1}',
 'Actemra US': '{Actemra2}',
 'Actemra Ex-US': '{Actemra1}-{Actemra2}',
 'Adempas WW': '{Adempas1}+{Adempas2}',
 'Adempas US': '{Adempas1}',
 'Adempas Ex-US': '{Adempas2}',
 'Aimovig WW': '{Aimovig1}',
 'Aimovig US': '{Aimovig2}',
 'Aimovig Ex-US': '{Aimovig1}-{Aimovig2}',
 'Aristada WW': '{Aristada1}',
 'Aristada US': '{Aristada1}',
 'Aristada Ex-US': '{Nan}',
 'Aubagio WW': '{Aubagio1}',
 'Aubagio US': '{Aubagio2}',
 'Aubagio Ex-US': '{Aubagio1}-{Aubagio2}',
 'Austedo WW': '{Austedo1}',
 'Austedo US': '{Austedo2}',
 'Austedo Ex-US': '{Austedo1}-{Austedo2}',
 'Biktarvy WW': '{Biktarvy1}',
 'Biktarvy US': '{Biktarvy2}',
 'Biktarvy Ex-US': '{Biktarvy1}-{Biktarvy2}',
 'Breo Ellipta WW': '{BreoEllipta1}',
 'Breo Ellipta US': '{BreoEllipta2}',
 'Breo Ellipta Ex-US': '{BreoEllipta1}-{BreoEllipta2}',
 'Brilinta WW': '{Brilinta1}',
 'Brilinta US': '{Brilinta2}',
 'Brilinta Ex-US': '{Brilinta1}-{Brilinta2}',
 'Calquence WW': '{Calquence1}',
 'Calquence US': '{Calqu

In [4]:
def evaluate_helper(clean_dataframe):
    '''
    '''
    holder = dict()
    for index, row in clean_dataframe.iterrows():
        for year in clean_dataframe.columns[2:]:
            if year not in holder:
                holder[year] = {row['Source'] : row[year]}
            else:
                temp = holder.get(year)
                holder[year].update({row['Source'] : row[year]})
                
    return holder

    
eval_map = evaluate_helper(clean)

def evaluate(eval_map, formula_key):
    '''
    '''
    return_dict = dict()
    for year in eval_map.keys():
        temp = eval_map.get(year)
        temp_dict = dict()
        for ID, formula in formula_key.items():
            if formula != '{Nan}':
                formula = formula.format(**temp)
                if formula != np.nan:
                    try:
                        result = eval(formula)
                    except ZeroDivisionError:
                        result = 0
                else:
                    result = formula
                temp_dict[ID] = result
            else:
                temp_dict[ID] = 'Nan'
                
        return_dict[year] = temp_dict
        
    return return_dict

evaluated_key = evaluate(eval_map, formula_key)


def turn_into_dataframe(evaluated_key):
    '''
    '''
    counter = 0 
    for year, values in evaluated_key.items():
        data = []
        for key, amount in values.items():
            data.append([key, amount])
        df_temp = pd.DataFrame(data, columns = ['Product Name', year])
        if counter > 0:
            df = df.merge(df_temp, how='inner', on='Product Name')
        else: 
            df = df_temp

        counter += 1
        
    return df
    
    
evaluated_key = turn_into_dataframe(evaluated_key)

In [5]:
def check_string(x, string_list):
    for string in string_list:
        if string in x:
            return string
    return None



def merge_final_clean(raw_dataframe, evaluated_key):
    '''
    '''
    cols_y = ['FY2007', 'FY2008', 'FY2009', 'FY2010', 'FY2011',
       'FY2012', 'FY2013', 'FY2014', 'FY2015', 'FY2016', 'FY2017', 'FY2018',
       'FY2019', 'FY2020', 'FY2021', 'FY2022', 'FY2023', 'FY2024', 'FY2025',
       'FY2026', 'FY2027', 'FY2028', 'FY2029', 'FY2030', 'FY2031', 'FY2032',
       'FY2033', 'FY2034', 'FY2035', 'FY2036', 'FY2037', 'FY2038']
    
    cols = ['Proper Name', 'Generic Name', 
            'Medicare Spend', 'Original Manufacturer', 
            'Application', 'Approval Date', 'Year']
    
    pnames = list(raw_dataframe['Proper Name'])
    temp = raw_dataframe[cols].drop_duplicates()
    
    evaluated_key["Proper Name"] = evaluated_key["Product Name"].apply(lambda x: check_string(x, pnames))
    
    final = temp.merge(evaluated_key, how='left', on='Proper Name')

    for col in cols_y:
        final = final.rename(columns={col: col[2:]})
    
    return final

    
final = merge_final_clean(df, evaluated_key)

In [24]:
final

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Application,Approval Date,Year,Product Name,2007,2008,...,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038
0,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra WW,0,0.0,...,1026.982709,950.614498,983.526357,1026.718156,1067.725867,1061.26059,1244.462648,NaN,NaN,NaN
1,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra US,0,0.0,...,335.725889,315.654785,335.01203,343.125495,362.802547,358.359781,363.288261,NaN,NaN,NaN
2,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra Ex-US,0,0.0,...,691.25682,634.959712,648.514326,683.592661,704.92332,702.900809,881.174387,NaN,NaN,NaN
3,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas WW,0,0.0,...,425.220348,383.039432,299.075119,182.487295,167.113971,90.296491,47.900762,NaN,NaN,NaN
4,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas US,0,0.0,...,238.420843,234.891855,204.027394,86.399891,71.240784,59.50577,26.347257,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi US,0,0.0,...,13.176228,5.622733,2.839458,1.589025,1.244972,0.882307,NaN,NaN,NaN,NaN
278,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi Ex-US,0,0.0,...,23.470813,19.440799,15.082767,12.319778,5.1098,4.200429,NaN,NaN,NaN,NaN
279,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy WW,0,0.0,...,1052.208894,859.211704,529.341355,451.13601,362.668629,615.523927,600.70167,NaN,NaN,NaN
280,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy US,0,0.0,...,608.303033,475.140206,287.319594,242.144549,159.085817,182.393814,145.915051,NaN,NaN,NaN


In [6]:
def reformat_final(dataframe):
    '''
    '''
    data = []
    years = dataframe.columns[8:]
    other_info = dataframe.columns[:8]
    
    for index, row in dataframe.iterrows():
        new_data = list(row[other_info].values)
        for year in years:
            #if row[str(year)] != 0:
                #new_data.append(row[str(year)])
            if int(year) >= row['Year']:
                new_data.append(row[str(year)])
                
        data.append(new_data)
        
    longest_list = max(data, key=lambda x: len(x))
    num_years = len(longest_list) - 8
    year_cols = [f't{i}' for i in range(1, num_years+1)]
    cols = list(other_info) + year_cols
        
    data = pd.DataFrame(data, columns = cols)
    data[year_cols] = data[year_cols].astype(float)
    data['t31'] = np.nan

    return data.fillna(np.nan)


re_final = reformat_final(final)
re_final

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Application,Approval Date,Year,Product Name,t1,t2,...,t22,t23,t24,t25,t26,t27,t28,t29,t30,t31
0,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra WW,561.764528,723.826320,...,629.456868,NaN,683.344555,679.206777,796.456095,NaN,NaN,NaN,NaN,NaN
1,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra US,82.071392,165.144840,...,214.407699,NaN,232.193630,229.350260,232.504487,NaN,NaN,NaN,NaN,NaN
2,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra Ex-US,479.693136,558.681480,...,415.049169,NaN,451.150925,449.856518,563.951608,NaN,NaN,NaN,NaN,NaN
3,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas WW,0.000000,149.229059,...,112.248170,57.620265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas US,0.000000,149.229059,...,81.457449,36.066760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi US,0.000000,159.675000,...,68.109291,48.268758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi Ex-US,0.000000,0.000000,...,279.544221,229.794826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy WW,473.760000,912.999200,...,451.136010,362.668629,615.523927,600.701670,NaN,NaN,NaN,NaN,NaN,NaN
280,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy US,425.068000,650.479600,...,242.144549,159.085817,182.393814,145.915051,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def find_last_valid(row):
    last_valid = row.last_valid_index()
    return (last_valid, row[last_valid])

def pro_rate(dataframe):
    '''
    '''
    last_values = dataframe.apply(find_last_valid, axis=1).tolist()
    
    end_of_year = pd.to_datetime(dataframe['Approval Date'].dt.year.astype(str) + '-12-31')
    days_remaining = (end_of_year - dataframe['Approval Date']).dt.days
    dataframe['percent_remaining'] = (days_remaining / 365)
    dataframe['percent_missing'] = 1 - dataframe['percent_remaining']
    
    for year in range(1,31):
        var = 't' + str(year)
        var1 = 't' + str(year+1)
        dataframe[var] = (dataframe[var]) + (dataframe[var1]*dataframe['percent_missing'])
        dataframe[var1] -= (dataframe[var1]*dataframe['percent_missing'])
    
    for i, (col, val) in enumerate(last_values):
        dataframe.loc[dataframe.index[i], col] = val
    
    return dataframe


pro_rate = pro_rate(re_final)
pro_rate

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Application,Approval Date,Year,Product Name,t1,t2,...,t24,t25,t26,t27,t28,t29,t30,t31,percent_remaining,percent_missing
0,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra WW,577.629214,730.395429,...,683.253864,681.776625,796.456095,NaN,NaN,NaN,NaN,NaN,0.978082,0.021918
1,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra US,85.691005,167.946302,...,232.131310,229.419393,232.504487,NaN,NaN,NaN,NaN,NaN,0.978082,0.021918
2,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra Ex-US,491.938210,562.449128,...,451.122554,452.357232,563.951608,NaN,NaN,NaN,NaN,NaN,0.978082,0.021918
3,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas WW,114.885933,257.876207,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.230137,0.769863
4,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas US,114.885933,228.904722,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.230137,0.769863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi US,106.304178,405.003786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334247,0.665753
278,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi Ex-US,0.000000,81.756649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334247,0.665753
279,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy WW,683.874884,985.102222,...,612.112777,600.701670,NaN,NaN,NaN,NaN,NaN,NaN,0.769863,0.230137
280,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy US,574.767415,668.967685,...,173.998701,145.915051,NaN,NaN,NaN,NaN,NaN,NaN,0.769863,0.230137


In [8]:
def apply_discount(dataframe, discount_rate=0.1):
    '''
    '''
    for year in range(1,31):
        var = 't' + str(year)
        dataframe[var] = dataframe[var]/(1+discount_rate)**(year-0.5)
        
    return dataframe

final = apply_discount(pro_rate)

In [13]:
year_cols = [f't{i}' for i in range(1, 30+1)]
final[final['Proper Name'] == 'Adempas'][year_cols[10:]]

,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21,t22,t23,t24,t25,t26,t27,t28,t29,t30
3,401.411200,377.481406,300.188185,236.489415,181.033948,124.390308,98.135354,74.516506,NaN,NaN,18.555209,9.043695,6.749006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285.900983,264.538311,192.212973,134.235028,101.018642,75.687117,65.548775,54.281746,NaN,NaN,12.068602,5.992804,4.224465,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,115.510217,112.943095,107.975212,102.254388,80.015306,48.703191,32.586579,20.234760,16.436865,14.954196,6.486607,3.050891,2.524541,NaN,NaN,NaN,NaN,NaN,NaN,NaN
